This notebook is designed to help me build Magic: The Gathering decks using external data from the website "Scryfall," which tracks datapoints on all cards, such as set, rarity, price, and more.

In building this, I hope to automate the process of budgeting for a new deck by calling Scryfall's API to get me prices.

I am also evaluating using a task scheduler to execute this process on a set schedule. This way, I could evaluate when a card price seems to be fluctuating, allowing me to make a decision about when I want to purchase it.

In [1]:
import requests as requests

In [2]:
# An initial exploration into Scryfall's "bulk" repository.

url = 'https://api.scryfall.com/bulk-data'
headers = {'Content-type': 'application/json'}
r = requests.get(url, headers=headers)
r.text

'{"object":"list","has_more":false,"data":[{"object":"bulk_data","id":"27bf3214-1271-490b-bdfe-c0be6c23d02e","type":"oracle_cards","updated_at":"2023-04-09T09:02:18.555+00:00","uri":"https://api.scryfall.com/bulk-data/27bf3214-1271-490b-bdfe-c0be6c23d02e","name":"Oracle Cards","description":"A JSON file containing one Scryfall card object for each Oracle ID on Scryfall. The chosen sets for the cards are an attempt to return the most up-to-date recognizable version of the card.","size":114387457,"download_uri":"https://data.scryfall.io/oracle-cards/oracle-cards-20230409090218.json","content_type":"application/json","content_encoding":"gzip"},{"object":"bulk_data","id":"6bbcf976-6369-4401-88fc-3a9e4984c305","type":"unique_artwork","updated_at":"2023-04-09T09:03:07.111+00:00","uri":"https://api.scryfall.com/bulk-data/6bbcf976-6369-4401-88fc-3a9e4984c305","name":"Unique Artwork","description":"A JSON file of Scryfall card objects that together contain all unique artworks. The chosen cards 

In [3]:
# Looks good!
r.status_code

200

This is where the magic (hah) happens. Now that we have a connection to the API, we can write these card entries to a Pandas dataframe. First, we load in our Excel spreadsheet. Then we us a "for" loop to iterate over the names in the spreadsheet.

Once we have names, we create a blank "prices" list, then generate URLs based on card name--this is accomplished using an F-string.

In the same "for" loop, we also query the API, retrieve a response in JSON, and append the 'price' column entry for that card.

Last, we output to a Pandas dataframe and check the results. Looking good!

In [ ]:
import requests
import openpyxl

# Load Excel file
wb = openpyxl.load_workbook('cards.xlsx')
ws = wb.active

# Create empty list to append results
card_data = []

# Iterate over rows and fetch prices
for row in ws.iter_rows(min_row=2, values_only=True):
    card_name = row[0]
    set_code = row[1]
    collector_number = row[2]

    # Define API query URL
    url = f"https://api.scryfall.com/cards/{set_code}/{collector_number}"

    # Standard API fetch
    response = requests.get(url)
    data = response.json()

    # Check for price.
    if data['object'] == 'error':
        # Queries fuzzy match on card name if not found by set code
        url = f"https://api.scryfall.com/cards/named?fuzzy={card_name}"
        response = requests.get(url)
        data = response.json()
        
        #Search fuzzy match with card name if set code + collector number returns an error
        if data['object'] == 'error':
            print(f"No data found for {card_name} ({set_code} {card_number}")
        else: price = data['prices']['usd']
        # print(f"{card_name} ({set_code} {card_number}): {price}")
        card_data.append([card_name, price])
               
    else:
        price = data['prices']['usd']
        #print(f"{card_name} ({set_code} {card_number}): {price}")
        card_data.append([card_name, price])

# Create a pandas dataframe and append all entries
df = pd.DataFrame(cards_data, columns=['Card Name', 'Price'])

df.head()        

Looking to export these files to CSV now. Let's take a look at the best option.

In [9]:
# Checking for working directory to see where results will be exported.
import os
print(os.getcwd())

C:\Users\twiss\Documents


In [10]:
# Export the complete CSV to a directory for reference. Separate using tabs to create separate columns.
df.to_csv('scryfall.csv', sep='\t', index=False)